# EcoMetricx — Retrieval Demo (MVP)

This notebook demos the early query system pipeline:
- Load normalized documents (Phase 2 output)
- If missing, auto-run normalization to generate it
- Chunk into retrieval units (simple, page-aware)
- Build a TF-IDF index (CPU-only)
- Run keyword/semantic-ish search and show citations

Run cells top-to-bottom to try queries.


In [1]:
from pathlib import Path
import json, subprocess, sys
from datetime import datetime

project_root = Path.cwd()
run_id_path = project_root / '.current_run_id'
run_id = run_id_path.read_text().strip() if run_id_path.exists() else None
print('Project root:', project_root)
print('Current run_id:', run_id)

Project root: /root/Programming Projects/Personal/EcoMetricx
Current run_id: 20250903_093826


In [2]:
# Ensure normalized documents exist; auto-run normalization if missing
norm_base = project_root / 'data' / 'normalized' / 'visual_extraction'
if run_id is None or not (norm_base / run_id / 'documents.jsonl').exists():
	print('Normalized documents missing; running normalization script...')
	ret = subprocess.run([sys.executable, str(project_root / 'scripts' / 'normalize_to_documents.py')], capture_output=True, text=True)
	print(ret.stdout or ret.stderr)
	# Refresh run_id if it was None
	if run_id is None and run_id_path.exists():
		run_id = run_id_path.read_text().strip()

norm_doc = norm_base / run_id / 'documents.jsonl'
assert norm_doc.exists(), f'Missing {norm_doc}'
rows = [json.loads(l) for l in norm_doc.read_text(encoding='utf-8').splitlines() if l.strip()]
print('Documents loaded:', len(rows))
print('Document id:', rows[0]['document_id'])


Documents loaded: 1
Document id: emx:visual_extraction:6a55e73ff2d9


## Create simple chunks (page-aware)
We split the document text by pages into retrieval units and attach page numbers and a section path placeholder.


In [3]:
chunk_file = project_root / 'data' / 'chunks' / 'visual_extraction' / run_id / 'chunks.jsonl'
if not chunk_file.exists():
	print('Chunks missing; running chunking script...')
	ret = subprocess.run([sys.executable, str(project_root / 'scripts' / 'chunk_and_redact.py')], capture_output=True, text=True)
	print(ret.stdout or ret.stderr)

chunks = [json.loads(l) for l in chunk_file.read_text(encoding='utf-8').splitlines() if l.strip()]
print('Chunks loaded:', len(chunks))
print('Sample chunk:', {k: chunks[0][k] for k in ('parent_document_id','page_num','section_path')})



Chunks loaded: 1
Sample chunk: {'parent_document_id': 'emx:visual_extraction:6a55e73ff2d9', 'page_num': 1, 'section_path': 'page/1'}


In [4]:
doc = rows[0]
chunks = []
for p in doc.get('pages', []):
	text = p.get('text','').strip()
	if not text:
		continue
	chunks.append({
		'chunk_index': len(chunks),
		'page_num': p.get('page_number', 0),
		'parent_document_id': doc['document_id'],
		'section_path': f"page/{p.get('page_number', 0)}",
		'text': text
	})
print('Chunks:', len(chunks))
print(chunks[0]['text'][:200] + ('...' if len(chunks[0]['text'])>200 else ''))


Chunks: 2
Home Energy Report: electricity March report Account number: 954137 Service address: 1627 Tulip Lane Dear JILL DOE, here is your usage analysis for March. Your electric use: 18% more than similar near...


## Build TF-IDF index
We use scikit-learn's `TfidfVectorizer` to index chunk texts for quick keyword/semantic-ish search.


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

corpus = [c['text'] for c in chunks]
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
X = vectorizer.fit_transform(corpus)
print('Index shape:', X.shape)



Index shape: (2, 135)


## Embeddings-based retrieval (BGE-small)
We use FastEmbed with `BAAI/bge-small-en-v1.5` to embed chunks and queries, then perform cosine similarity search. Falls back to TF-IDF if unavailable.


In [6]:
# Ensure embeddings exist; auto-generate if missing (auto-install fastembed)
from importlib import util as _iu

emb_dir = project_root / 'data' / 'index' / 'pgvector' / run_id
emb_file = emb_dir / 'embeddings.jsonl'
manifest_file = emb_dir / 'index_manifest.json'

# Ensure fastembed is available in this kernel
if _iu.find_spec('fastembed') is None:
	print('Installing fastembed in current kernel...')
	_ = subprocess.run([sys.executable, '-m', 'pip', 'install', 'fastembed', '--quiet'], text=True)

# Generate embeddings if missing
if not emb_file.exists():
	print('Embeddings missing; generating...')
	ret = subprocess.run([sys.executable, str(project_root / 'scripts' / 'embed_chunks.py')], capture_output=True, text=True)
	print(ret.stdout or ret.stderr)

if emb_file.exists():
	print('Embeddings ready:', emb_file)
	manifest = json.loads(manifest_file.read_text())
	print('Model:', manifest.get('model'), 'dim:', manifest.get('embedding_dim'))
else:
	print('Embeddings not available; search will use TF-IDF fallback.')


Embeddings ready: /root/Programming Projects/Personal/EcoMetricx/data/index/pgvector/20250903_093826/embeddings.jsonl
Model: BAAI/bge-small-en-v1.5 dim: 384


In [7]:
# Build in-memory embedding matrix if available
import numpy as np

emb_records = []
if emb_file.exists():
	for line in emb_file.read_text().splitlines():
		if not line.strip():
			continue
		r = json.loads(line)
		emb_records.append(r)
	E = np.vstack([np.array(r['embedding_vector'], dtype=np.float32) for r in emb_records]) if emb_records else None
	id_to_idx = {r['chunk_id']: i for i, r in enumerate(emb_records)}
	print('Embedding matrix:', E.shape if E is not None else None)
else:
	E = None
	id_to_idx = {}



Embedding matrix: (1, 384)


## Normalize chunk IDs (safety)
Ensure every chunk has a `chunk_id` and build a lookup by id. This prevents KeyError if earlier cells created temporary chunks without IDs.


In [8]:
# Ensure each chunk has a stable chunk_id and build lookup
for i, c in enumerate(chunks):
	if 'chunk_id' not in c:
		c['chunk_id'] = f"{c['parent_document_id']}:c{c.get('chunk_index', i)}"
chunk_by_id = {c['chunk_id']: c for c in chunks}
print('Unique chunk ids:', len(chunk_by_id))



Unique chunk ids: 2


In [9]:
# Override search_embedded to use chunk_by_id
import numpy as np

def search_embedded(query: str, k: int = 3):
	if E is None:
		print('Embeddings not loaded; falling back to TF-IDF search()')
		return search(query, k)
	from fastembed import TextEmbedding
	emb = TextEmbedding('BAAI/bge-small-en-v1.5')
	qv = np.array(list(emb.embed([query]))[0], dtype=np.float32)
	qv = qv / (np.linalg.norm(qv) + 1e-9)
	Ev = E / (np.linalg.norm(E, axis=1, keepdims=True) + 1e-9)
	scores = Ev @ qv
	idxs = scores.argsort()[-k:][::-1]
	results = []
	for idx in idxs:
		chunk_id = emb_records[idx]['chunk_id']
		c = chunk_by_id.get(chunk_id)
		if not c:
			continue
		results.append({
			'score': float(scores[idx]),
			'document_id': c['parent_document_id'],
			'page_num': c['page_num'],
			'section_path': c['section_path'],
			'snippet': c['text'][:240] + ('...' if len(c['text'])>240 else '')
		})
	return results

# Demo
for r in search_embedded('energy savings tips'):
	print(r['score'], r['document_id'], f"page {r['page_num']}")
	print(r['snippet'])
	print('---')



0.7733845710754395 emx:visual_extraction:6a55e73ff2d9 page 0
Home Energy Report: electricity March report Account number: 954137 Service address: 1627 Tulip Lane Dear JILL DOE, here is your usage analysis for March. Your electric use: 18% more than similar nearby homes You TT A bove Similar nearby ho...
---


In [10]:
# Backfill chunk_id if missing and build map
for i, c in enumerate(chunks):
	if 'chunk_id' not in c:
		c['chunk_id'] = f"{c['parent_document_id']}:c{c.get('chunk_index', i)}"
chunk_by_id = {c['chunk_id']: c for c in chunks}
print('Unique chunk ids:', len(chunk_by_id))



Unique chunk ids: 2


In [11]:
# Embedding-backed search; falls back to TF-IDF

def search_embedded(query: str, k: int = 3):
	if E is None:
		print('Embeddings not loaded; falling back to TF-IDF search()')
		return search(query, k)
	from fastembed import TextEmbedding
	emb = TextEmbedding('BAAI/bge-small-en-v1.5')
	qv = np.array(list(emb.embed([query]))[0], dtype=np.float32)
	# cosine similarity
	qv = qv / (np.linalg.norm(qv) + 1e-9)
	Ev = E / (np.linalg.norm(E, axis=1, keepdims=True) + 1e-9)
	scores = Ev @ qv
	idxs = scores.argsort()[-k:][::-1]
	results = []
	for idx in idxs:
		chunk_id = emb_records[idx]['chunk_id']
		c = next(c for c in chunks if c['chunk_id'] == chunk_id)
		results.append({
			'score': float(scores[idx]),
			'document_id': c['parent_document_id'],
			'page_num': c['page_num'],
			'section_path': c['section_path'],
			'snippet': c['text'][:240] + ('...' if len(c['text'])>240 else '')
		})
	return results

# Demo
for r in search_embedded('energy savings tips'):
	print(r['score'], r['document_id'], f"page {r['page_num']}")
	print(r['snippet'])
	print('---')



0.7733845710754395 emx:visual_extraction:6a55e73ff2d9 page 0
Home Energy Report: electricity March report Account number: 954137 Service address: 1627 Tulip Lane Dear JILL DOE, here is your usage analysis for March. Your electric use: 18% more than similar nearby homes You TT A bove Similar nearby ho...
---


In [12]:
def search(query: str, k: int = 3):
	qv = vectorizer.transform([query])
	scores = cosine_similarity(qv, X)[0]
	topk = scores.argsort()[::-1][:k]
	results = []
	for idx in topk:
		c = chunks[idx]
		results.append({
			'score': float(scores[idx]),
			'document_id': c['parent_document_id'],
			'page_num': c['page_num'],
			'section_path': c['section_path'],
			'snippet': c['text'][:240] + ('...' if len(c['text'])>240 else '')
		})
	return results

for r in search('energy savings tips'):
	print(r['score'], r['document_id'], f"page {r['page_num']}")
	print(r['snippet'])
	print('---')



0.4048466114956216 emx:visual_extraction:6a55e73ff2d9 page 1
Your top three tailored energy-saving tips Caulk windows and doors Upgrade your refrigerator Adjust thermostat settings Save money and energy Look for an Energy Star label Biggest energy saving option One of the biggest Older model Set your...
---
0.12844813733286536 emx:visual_extraction:6a55e73ff2d9 page 0
Home Energy Report: electricity March report Account number: 954137 Service address: 1627 Tulip Lane Dear JILL DOE, here is your usage analysis for March. Your electric use: 18% more than similar nearby homes You TT A bove Similar nearby ho...
---


## Optional: Ingest current run into Postgres (Phase 5)
This cell applies the migration and ingests documents, chunks, and embeddings into Postgres using `DATABASE_URL`. Requires pgvector extension installed.


In [13]:
# Ingest to Postgres using DATABASE_URL
import os
from importlib import util as _iu

# Ensure psycopg and dotenv available in kernel
missing = []
for m in ('psycopg', 'dotenv'):
	if _iu.find_spec(m) is None:
		missing.append(m)
if missing:
	print('Installing missing packages:', missing)
	_ = subprocess.run([sys.executable, '-m', 'pip', 'install', *missing, '--quiet'], text=True)

# Apply migration (requires psql available). Skip if not present.
DATABASE_URL = os.environ.get('DATABASE_URL') or os.environ.get('POSTGRES_DSN')
if DATABASE_URL:
	print('Using DATABASE_URL')
	# Best effort migration via psql if available
	psql = subprocess.run(['which', 'psql'], capture_output=True, text=True)
	if psql.returncode == 0:
		print('Applying migration 001_init.sql')
		_ = subprocess.run(['psql', DATABASE_URL, '-f', str(project_root / 'db' / 'migrations' / '001_init.sql')], text=True)
	else:
		print('psql not found; please apply db/migrations/001_init.sql manually')
	# Ingest
	print('Ingesting current run...')
	ret = subprocess.run([sys.executable, str(project_root / 'scripts' / 'ingest_to_postgres.py')], capture_output=True, text=True)
	print(ret.stdout or ret.stderr)
else:
	print('DATABASE_URL not set; skipping ingestion')



Using DATABASE_URL
Applying migration 001_init.sql


psql:/root/Programming Projects/Personal/EcoMetricx/db/migrations/001_init.sql:2: ERROR:  extension "vector" is not available
DETAIL:  Could not open extension control file "/usr/share/postgresql/17/extension/vector.control": No such file or directory.
HINT:  The extension must first be installed on the system where PostgreSQL is running.
psql:/root/Programming Projects/Personal/EcoMetricx/db/migrations/001_init.sql:21: NOTICE:  relation "documents" already exists, skipping
psql:/root/Programming Projects/Personal/EcoMetricx/db/migrations/001_init.sql:40: NOTICE:  relation "chunks" already exists, skipping


CREATE TABLE
CREATE TABLE


psql:/root/Programming Projects/Personal/EcoMetricx/db/migrations/001_init.sql:48: ERROR:  type "vector" does not exist
LINE 4:   embedding VECTOR(384) NOT NULL
                    ^
psql:/root/Programming Projects/Personal/EcoMetricx/db/migrations/001_init.sql:52: NOTICE:  column "text_tsv" of relation "chunks" already exists, skipping
psql:/root/Programming Projects/Personal/EcoMetricx/db/migrations/001_init.sql:54: NOTICE:  relation "idx_chunks_text_tsv" already exists, skipping


ALTER TABLE
CREATE INDEX
CREATE FUNCTION
DROP TRIGGER
CREATE TRIGGER
Ingesting current run...


psql:/root/Programming Projects/Personal/EcoMetricx/db/migrations/001_init.sql:77: ERROR:  relation "chunk_embeddings" does not exist
CONTEXT:  SQL statement "CREATE INDEX IF NOT EXISTS idx_chunk_embeddings_ivf ON chunk_embeddings USING ivfflat (embedding vector_cosine_ops) WITH (lists = 100)"
PL/pgSQL function inline_code_block line 6 at EXECUTE


✅ Ingested documents and chunks (FTS-only)



## Call the Retrieval API (optional)
This cell demonstrates calling the local FastAPI service (`/search`). Set `API_KEY` to your generated key and ensure the server is running.


In [14]:
import os, requests, json as _json
API_HOST = os.environ.get('API_HOST', 'http://127.0.0.1:8000')
API_KEY = os.environ.get('API_KEY', 'REPLACE_THIS')  # replace with your key

payload = {"query": "energy savings tips", "k": 3}
headers = {"X-API-Key": API_KEY, "Content-Type": "application/json"}
try:
	resp = requests.post(f"{API_HOST}/search", headers=headers, data=_json.dumps(payload), timeout=10)
	print(resp.status_code)
	print(resp.json())
except Exception as e:
	print('API call failed:', e)



API call failed: HTTPConnectionPool(host='127.0.0.1', port=8000): Max retries exceeded with url: /search (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fc9611347d0>: Failed to establish a new connection: [Errno 111] Connection refused'))


## Load .env for DB ingestion
Set environment variables from your `.env` so the ingestion cell can read `DATABASE_URL`.


In [15]:
import os
try:
	from dotenv import load_dotenv
	exists = load_dotenv()
	print('Loaded .env:', exists)
	print('DATABASE_URL set:', bool(os.environ.get('DATABASE_URL')))
except Exception as e:
	print('dotenv not available or failed:', e)



Loaded .env: True
DATABASE_URL set: True


## Query the Retrieval API from this notebook
Set `API_HOST` and `API_KEY` (or keep defaults if you exported them to the environment). The cells below call `/search` and `/similar`.


In [16]:
import os, json, requests
API_HOST = os.environ.get('API_HOST', 'http://127.0.0.1:8000')
API_KEY = os.environ.get('API_KEY', '')  # set here if not in env

print('API_HOST:', API_HOST)
print('API_KEY set:', bool(API_KEY))

payload = {"query": "energy savings tips", "k": 3}
headers = {"X-API-Key": API_KEY, "Content-Type": "application/json"}
try:
	resp = requests.post(f"{API_HOST}/search", headers=headers, data=json.dumps(payload), timeout=10)
	print(resp.status_code)
	print(resp.json())
except Exception as e:
	print('API call failed:', e)



API_HOST: http://127.0.0.1:8000
API_KEY set: True
API call failed: HTTPConnectionPool(host='127.0.0.1', port=8000): Max retries exceeded with url: /search (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fc96383d010>: Failed to establish a new connection: [Errno 111] Connection refused'))


In [17]:
similar_payload = {"chunk_id": "emx:visual_extraction:6a55e73ff2d9:c0", "k": 5}
headers = {"X-API-Key": API_KEY, "Content-Type": "application/json"}
try:
	resp = requests.post(f"{API_HOST}/similar", headers=headers, data=json.dumps(similar_payload), timeout=10)
	print(resp.status_code)
	print(resp.json())
except Exception as e:
	print('API call failed:', e)



API call failed: HTTPConnectionPool(host='127.0.0.1', port=8000): Max retries exceeded with url: /similar (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fc95e575290>: Failed to establish a new connection: [Errno 111] Connection refused'))


## Interactive query (type your question)
Use the widget below to enter your query and hit Search. Make sure the API server is running and `API_KEY` is set.


In [18]:
import os, json, requests
from ipywidgets import Text, IntSlider, Button, VBox, HBox, Output
from IPython.display import display

API_HOST = os.environ.get('API_HOST', 'http://127.0.0.1:8000')
API_KEY = os.environ.get('API_KEY', '')

q_input = Text(description='Query:', placeholder='Type your question...')
k_input = IntSlider(description='Top-K', min=1, max=10, value=3)
run_btn = Button(description='Search', button_style='primary')
out = Output()

headers = {"X-API-Key": API_KEY, "Content-Type": "application/json"}

def on_click(_):
	out.clear_output()
	payload = {"query": q_input.value, "k": int(k_input.value)}
	try:
		resp = requests.post(f"{API_HOST}/search", headers=headers, data=json.dumps(payload), timeout=15)
		with out:
			print('Status:', resp.status_code)
			print(json.dumps(resp.json(), indent=2))
	except Exception as e:
		with out:
			print('API call failed:', e)

run_btn.on_click(on_click)

display(VBox([HBox([q_input, k_input, run_btn]), out]))



## API diagnostics and bulk query tests
Use these cells to inspect the API config and run a batch of common queries. Start the API and ensure `API_KEY`/`API_HOST` are set.


In [24]:
import os, json, requests
API_HOST = os.environ.get('API_HOST', 'http://127.0.0.1:8000')
API_KEY = os.environ.get('API_KEY', '')
headers = {"X-API-Key": API_KEY, "Content-Type": "application/json"}

# Call the new /debug/config endpoint added by Claude Code
try:
	resp = requests.get(f"{API_HOST}/debug/config", headers=headers, timeout=10)
	print('Status:', resp.status_code)
	print(json.dumps(resp.json(), indent=2))
except Exception as e:
	print('API call failed:', e)



Status: 200
{
  "fastembed_ok": true,
  "qdrant_configured": true,
  "qdrant_warm": true,
  "qdrant_points": 1,
  "documents": 1,
  "chunks": 1,
  "tsv_ready": 1
}


In [26]:
# Bulk query test using the updated fusion logic
queries = [
	"home energy report",
	"energy savings tips",
	"monthly savings tip",
	"thermostat settings advice",
	"caulk windows doors",
	"upgrade refrigerator",
	"usage compared similar homes",
	"march report",
]

non_empty = 0
for q in queries:
	payload = {"query": q, "k": 5}
	resp = requests.post(f"{API_HOST}/search", headers=headers, data=json.dumps(payload), timeout=15)
	print(f"Query: {q}")
	print('Status:', resp.status_code)
	try:
		res = resp.json()
		print('Count:', len(res.get('results', [])))
		if res.get('results'):
			non_empty += 1
		print(json.dumps(res, indent=2))
	except Exception:
		print(resp.text)
	print('-' * 60)

print('Non-empty queries:', non_empty, '/', len(queries))



Query: home energy report
Status: 200
Count: 1
{
  "results": [
    {
      "chunk_id": "emx:visual_extraction:6a55e73ff2d9:c0",
      "document_id": "emx:visual_extraction:6a55e73ff2d9",
      "page_num": 1,
      "score": 0.7200000400000001,
      "snippet": "Your top three tailored energy-saving tips Caulk windows and doors Upgrade your refrigerator Adjust thermostat settings Save money and energy Look for an Energy Star label Biggest energy saving option One of the biggest Older model Set your smart \\ money-wasters in | refrigerators are thermostat to ..."
    }
  ]
}
------------------------------------------------------------
Query: energy savings tips
Status: 200
Count: 1
{
  "results": [
    {
      "chunk_id": "emx:visual_extraction:6a55e73ff2d9:c0",
      "document_id": "emx:visual_extraction:6a55e73ff2d9",
      "page_num": 1,
      "score": 0.04529914560000001,
      "snippet": "Your top three tailored energy-saving tips Caulk windows and doors Upgrade your refrigerator Ad